In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics, model_selection

In [2]:
orderProductsTrain = pd.read_csv("order_products__train.csv")
orderProductsPrior = pd.read_csv("order_products__prior.csv")
orders = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")
aisles = pd.read_csv("aisles.csv")
departments = pd.read_csv("departments.csv")

In [3]:
orderProductsAll = pd.concat([orderProductsTrain, orderProductsPrior], axis=0)
total = orderProductsAll.isnull().sum().sort_values(ascending=False)
percentage = (orderProductsAll.isnull().sum()/orderProductsAll.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percentage], axis=1, keys=['Total Missing', 'Percent'])
missing_data

,Total Missing,Percent
reordered,0,0.0
add_to_cart_order,0,0.0
product_id,0,0.0
order_id,0,0.0


In [4]:
grouped = orderProductsAll.groupby("reordered")["product_id"].aggregate({'Total_products': 'count'}).reset_index()
grouped['Ratios'] = grouped["Total_products"].apply(lambda x: x /grouped['Total_products'].sum())
grouped

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  if __name__ == '__main__':


,reordered,Total_products,Ratios
0,0,13863746,0.409938
1,1,19955360,0.590062


59% of products are ordered by prior customers

In [5]:
ordersdf = pd.read_csv("orders.csv", usecols=["order_id","user_id","order_number"])

In [6]:
priordf = pd.read_csv("order_products__prior.csv")

# merge with the orders file to get the user_id #
priordf = pd.merge(priordf, ordersdf, how="inner", on="order_id")
priordf.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number
0,2,33120,1,1,202279,3
1,2,28985,2,1,202279,3
2,2,9327,3,0,202279,3
3,2,45918,4,1,202279,3
4,2,30035,5,0,202279,3


In [7]:
# get the products and reorder status of the latest purchase of each user #
priorGroupeddf = priordf.groupby("user_id")["order_number"].aggregate("max").reset_index()
priordf_latest = pd.merge(priordf, priorGroupeddf, how="inner", on=["user_id", "order_number"])
priordf_latest = priordf_latest[["user_id", "product_id", "reordered"]]
priordf_latest.columns = ["user_id", "product_id", "reordered_latest"]
priordf_latest.head()

,user_id,product_id,reordered_latest
0,59897,9755,1
1,59897,31487,0
2,59897,37510,1
3,59897,14576,1
4,59897,22105,0


In [8]:
# get the count of each product and number of reorders by the customer #
priordfAgg = priordf.groupby(["user_id","product_id"])["reordered"].aggregate(["count", "sum"]).reset_index()
priordfAgg.columns = ["user_id", "product_id", "reordered_count", "reordered_sum"]
priordfAgg.head()

,user_id,product_id,reordered_count,reordered_sum
0,1,196,10,9
1,1,10258,9,8
2,1,10326,1,0
3,1,12427,10,9
4,1,13032,3,2


In [9]:
priordfMerge = pd.merge(priordfAgg, priordf_latest, how="left", on=["user_id","product_id"])
priordfMerge.head()

,user_id,product_id,reordered_count,reordered_sum,reordered_latest
0,1,196,10,9,1.0
1,1,10258,9,8,1.0
2,1,10326,1,0,NaN
3,1,12427,10,9,1.0
4,1,13032,3,2,1.0


In [10]:
ordersdf.drop(["order_number"],axis=1,inplace=True)

train_df = pd.read_csv("order_products__train.csv", usecols=["order_id"])
train_df.head()

,order_id
0,1
1,1
2,1
3,1
4,1


In [11]:
train_df = train_df.groupby("order_id").aggregate("count").reset_index()
train_df.head()

,order_id
0,1
1,36
2,38
3,96
4,98


In [12]:
train_df = pd.merge(train_df, ordersdf, how="inner", on="order_id")
train_df.head()

,order_id,user_id
0,1,112108
1,36,79431
2,38,42756
3,96,17227
4,98,56463


In [13]:
test_df = pd.read_csv("sample_submission.csv", usecols=["order_id"])

test_df = pd.merge(test_df, ordersdf, how="inner", on="order_id")
test_df.head()

,order_id,user_id
0,17,36855
1,34,35220
2,137,187107
3,182,115892
4,257,35581


In [14]:
train_df = pd.merge(train_df, priordfMerge, how="inner", on="user_id")
train_df.head()

,order_id,user_id,product_id,reordered_count,reordered_sum,reordered_latest
0,1,112108,2067,1,0,NaN
1,1,112108,5707,2,1,1.0
2,1,112108,11109,2,1,NaN
3,1,112108,14947,3,2,1.0
4,1,112108,22035,2,1,NaN


In [15]:
test_df = pd.merge(test_df, priordfMerge, how="inner", on="user_id")
#del prior_df, prior_grouped_df, prior_df_latest
test_df.head()

,order_id,user_id,product_id,reordered_count,reordered_sum,reordered_latest
0,17,36855,1283,1,0,NaN
1,17,36855,6291,1,0,NaN
2,17,36855,7035,1,0,NaN
3,17,36855,11494,1,0,NaN
4,17,36855,13107,3,2,1.0


In [16]:
productsdf = pd.read_csv("products.csv", usecols=["product_id", "aisle_id", "department_id"])
productsdf.head()

,product_id,aisle_id,department_id
0,1,61,19
1,2,104,13
2,3,94,7
3,4,38,1
4,5,5,13


In [17]:
train_df = pd.merge(train_df, productsdf, how="inner", on="product_id")
train_df.head()

,order_id,user_id,product_id,reordered_count,reordered_sum,reordered_latest,aisle_id,department_id
0,1,112108,2067,1,0,NaN,3,19
1,96,17227,2067,1,0,NaN,3,19
2,3243,206024,2067,1,0,NaN,3,19
3,12950,11456,2067,1,0,0.0,3,19
4,17683,177724,2067,1,0,NaN,3,19


In [18]:
test_df = pd.merge(test_df, productsdf, how="inner", on="product_id")
test_df.head()

,order_id,user_id,product_id,reordered_count,reordered_sum,reordered_latest,aisle_id,department_id
0,17,36855,1283,1,0,NaN,100,21
1,657743,16994,1283,1,0,NaN,100,21
2,834223,92628,1283,1,0,NaN,100,21
3,1446886,75870,1283,1,0,NaN,100,21
4,1518981,118458,1283,3,2,NaN,100,21


In [19]:
train_y_df = pd.read_csv("order_products__train.csv", usecols=["order_id", "product_id", "reordered"])
train_y_df = pd.merge(train_y_df, ordersdf, how="inner", on="order_id")
train_y_df = train_y_df[["user_id", "product_id", "reordered"]]
train_df = pd.merge(train_df, train_y_df, how="left", on=["user_id", "product_id"])
train_df["reordered"].fillna(0, inplace=True)
print(train_df.shape)

(8474661, 9)


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [21]:
X=train_df.drop('reordered',axis=1)

In [22]:
Y =train_df['reordered']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=7)

In [ ]:
from tpot import TPOTClassifier

tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_insta_pipeline.py')

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Imputing missing values in feature set
